In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline

We are curious about the relationships between different leading indicators to the lagging indicator (Fare)

We can only use train data to investigate the relationships

### Exploratory Data Analysis

In [ ]:
train = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv',nrows=1000000)
train.head()

In [ ]:
train.shape

#### Data Cleaning

1. data not make sense

2. null values

3. outliers

In [ ]:
#fare amount should be larger than 0 
train = train[(train['fare_amount'] >= 0)&(train['fare_amount'] <= 200)]

In [ ]:
#There should not be missing values
#Here we can see there is not any missing value
train.dropna(inplace=True)
train.isna().sum()

In [ ]:
#outliers
#First get descriptive stats
train.describe()

From here we can see pick_up_longitude and pickup_latitude actually have a small IQR. It is likely there are some wrong or outliers here. Let us visualize

In [ ]:
train = train[(train['pickup_longitude'] < -72)&(train['pickup_longitude'] > -75)]
train = train[(train['pickup_latitude'] < 44)&(train['pickup_latitude'] > 37)]
train = train[(train['dropoff_longitude'] < -72)&(train['dropoff_longitude'] > -75)]
train = train[(train['dropoff_latitude'] > 37) & (train['dropoff_latitude'] < 44)]
train = train[(train['passenger_count'] < 10)&(train['passenger_count'] > 0)]

In [ ]:
def finding_distance(df):
    R = 6373.0
    lat1 =np.asarray(np.radians(df['pickup_latitude']))
    lon1 = np.asarray(np.radians(df['pickup_longitude']))
    lat2 = np.asarray(np.radians(df['dropoff_latitude']))
    lon2 = np.asarray(np.radians(df['dropoff_longitude']))

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    ls1=[] 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    
    df['Distance']=np.asarray(distance)*0.621

finding_distance(train)
train = train[train['Distance'] < 50]

In [ ]:
#Distributution of drop off Longitude
sns.histplot(train['dropoff_longitude'],binwidth = 0.5)

In [ ]:
#Distributution of drop off Latitude
sns.histplot(train['dropoff_latitude'],binwidth = 0.5)

In [ ]:
#Distributution of Longitude
sns.histplot(train['pickup_longitude'],binwidth = 0.5)

In [ ]:
#Distributution of Latitude
sns.histplot(train['pickup_latitude'],binwidth = 0.5)


Get rid of the right part (These are outliers)

In [ ]:
round(train.describe(),2)

**From the decription statistics above**, everything looks clean now

#### Visualization of EDA


#### Timing and Fare

We think timing (different days in week, different months) may be a leading factor for different price

Firstly, get different parts of times as new columns

In [ ]:
train["date_parsed"] = pd.to_datetime(train["pickup_datetime"])

train["Year"] = train["date_parsed"].dt.year

train["Month"] = train["date_parsed"].dt.month

train["Weekday"] = train["date_parsed"].dt.weekday

train["hour"] = train["date_parsed"].dt.hour

train = train.drop(columns = ["key","pickup_datetime", "date_parsed"])

**Note**, this would be the new data frame we use to research the relationship between timing and fare

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(15,7))
sns.boxplot(x = train["Weekday"], y = train["fare_amount"])
plt.xlabel('Weekday')
plt.ylabel('Fare distribution')

**Surprisingly**, from the plot, weekday seems not to influence the fare amount significantly

In [ ]:
#Visualize the relationship between month and fare
plt.figure(figsize=(15,7))
sns.boxplot(x = train["Month"], y = train["fare_amount"])
plt.xlabel('Month')
plt.ylabel('Fare distribution')

The influence is also not significant!

In [ ]:
#Visualize the relationship between hour and fare
plt.figure(figsize=(15,7))
sns.boxplot(x = train["hour"], y = train["fare_amount"])
plt.xlabel('Hour')
plt.ylabel('Fare distribution')

This suggests some relationship. The boxplot suggests 4-5am have a relatively high overall fare amount. Why does this happen? we can see the frequency of the observations

In [ ]:
#Visualize the density of hour
plt.figure(figsize=(15,7))
sns.countplot(train["hour"])

In [ ]:
train.head()

In [ ]:
sns.histplot(train['passenger_count'],binwidth = 1)


In [ ]:
sns.histplot(train['Distance'],binwidth = 1)

In [ ]:
plt.figure(figsize=(15,7))
sns.boxplot(x = train["passenger_count"], y = train["fare_amount"])
plt.xlabel('passenger_count')
plt.ylabel('Fare distribution')

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(train["passenger_count"])

In [ ]:
train_df=train


In [ ]:
train_df.head()

In [ ]:

from sklearn.linear_model import LinearRegression
# lr=LinearRegression(normalize=True)
lr=LinearRegression()
from sklearn.model_selection import train_test_split
X=train_df.drop(['fare_amount','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','Year','Month','Weekday'],axis=1)
y=train_df['fare_amount']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01,random_state=80)
reg=lr.fit(X_train.values,y_train.values)
reg.intercept_

In [ ]:
print(X.head())

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
y_pred = reg.predict(X_test.values)
# The coefficients
print("Coefficients: \n", reg.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred,squared=False))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

In [ ]:
test = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')
train=test
train = train[(train['pickup_longitude'] < -72)&(train['pickup_longitude'] > -75)]
train = train[(train['pickup_latitude'] < 44)&(train['pickup_latitude'] > 37)]
train = train[(train['dropoff_longitude'] < -72)&(train['dropoff_longitude'] > -75)]
train = train[(train['dropoff_latitude'] > 37) & (train['dropoff_latitude'] < 44)]
train = train[(train['passenger_count'] < 10)&(train['passenger_count'] > 0)]
train["date_parsed"] = pd.to_datetime(train["pickup_datetime"])

train["Year"] = train["date_parsed"].dt.year

train["Month"] = train["date_parsed"].dt.month

train["Weekday"] = train["date_parsed"].dt.weekday

train["hour"] = train["date_parsed"].dt.hour

train = train.drop(columns = ["key","pickup_datetime", "date_parsed"])
finding_distance(train)
X=train.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','Year','Month','Weekday'],axis=1)
y_pred_test= reg.predict(X.values)

In [ ]:
X['pre_fare']=y_pred_test
X

You cann see there are smallest number of taxis between 4-5 am. This might suggest an overall high fee during this period